In [73]:
import polars as pl
pl.Config.set_tbl_rows(50)
pl.Config.set_tbl_cols(-1)

best_transformer_version = "H32_L3"
best_zoo_version = "H64_L1"

zoo_results_df = (
    pl.read_parquet(f"../data/models/zoo/{best_zoo_version}/model_preds.parquet")
    .rename({"tackle_x_pred": "tackle_x_rel_pred_zoo", "tackle_y_pred": "tackle_y_rel_pred_zoo"})
)
trfm_results_df = (
    pl.read_parquet(f"../data/models/transformer/{best_transformer_version}/model_preds.parquet")
    .rename({"tackle_x_pred": "tackle_x_rel_pred_trfm", "tackle_y_pred": "tackle_y_rel_pred_trfm"})
)

results_df = (
    trfm_results_df
    .join(
        zoo_results_df.select(["gameId", "playId", "mirrored", "frameId", "tackle_x_rel_pred_zoo", "tackle_y_rel_pred_zoo"]),
        on=["gameId", "playId", "mirrored", "frameId"],
        how="inner"
    ).with_columns(
        frame_distance_from_tackle = (pl.col("tackle_frameId") - pl.col("frameId")).cut(range(-10, 51, 5)),
    )

)

# results_df = (
#     pl.read_parquet(results_path)
#     .with_columns(
#         frame_distance_from_tackle = (pl.col("tackle_frameId") - pl.col("frameId")).cut(range(-10, 51, 5)),
#     )
# )

results_df.sample(3)

gameId,playId,mirrored,frameId,tackle_x_rel_pred_trfm,tackle_y_rel_pred_trfm,dataset_split,ballCarrierNflId,ballCarrierName,tackle_frameId,tackle_event,tackle_x,tackle_y,tackle_x_rel,tackle_y_rel,tackle_event_enum,tackle_x_rel_pred_zoo,tackle_y_rel_pred_zoo,frame_distance_from_tackle
i64,i64,bool,i64,f32,f32,str,i64,str,i64,str,f64,f64,f64,f64,i64,f32,f32,cat
2022110601,2616,true,18,10.05,2.03,"""train""",46377,"""Jeffery Wilson""",35,"""tackle""",78.75,53.02,11.38,5.91,0,6.01,7.86,"""(15, 20]"""
2022092507,1862,true,48,18.360001,0.72,"""test""",47807,"""Josh Jacobs""",60,"""tackle""",59.56,31.96,20.19,2.46,0,16.08,3.31,"""(10, 15]"""
2022100900,2817,false,5,11.89,3.56,"""train""",45186,"""Matt Breida""",36,"""tackle""",79.7,27.86,8.57,4.19,0,11.35,3.5,"""(30, 35]"""


In [74]:
# from sklearn.metrics import mean_squared_error
# from torch.nn.functional import mse_loss
# from torch import tensor
import numpy as np


def calculate_mse(x: pl.Series, y: pl.Series, xhat: pl.Series, yhat: pl.Series):
    """
    Calculate the mean squared error between the predicted and true values of x and y.
    """
    x, y, xhat, yhat = x.to_numpy(), y.to_numpy(), xhat.to_numpy(), yhat.to_numpy()
    return np.mean((np.array([xhat - x, yhat - y]) ** 2).mean(axis=0))


(
    results_df
    .group_by(["dataset_split"], maintain_order=True)
    .agg(
        zoo_mse = pl.map_groups(
                exprs=["tackle_x_rel", "tackle_y_rel", "tackle_x_rel_pred_zoo", "tackle_y_rel_pred_zoo"],
                function=lambda list_of_series: calculate_mse(*list_of_series),
                returns_scalar=True,
            ).round(1),
        trfm_mse = pl.map_groups(
                exprs=["tackle_x_rel", "tackle_y_rel", "tackle_x_rel_pred_trfm", "tackle_y_rel_pred_trfm"],
                function=lambda list_of_series: calculate_mse(*list_of_series),
                returns_scalar=True,
            ).round(1),
    ).with_columns(
        trfm_perc_adv = ((pl.col("zoo_mse") - pl.col("trfm_mse"))*100 / pl.col("zoo_mse")).round(1)
    )
)

dataset_split,zoo_mse,trfm_mse,trfm_perc_adv
str,f64,f64,f64
"""train""",36.1,31.1,13.9
"""val""",41.6,34.7,16.6
"""test""",38.0,30.7,19.2


In [75]:
test_loss_by_frame_df = (
    results_df
    .filter(pl.col("dataset_split") == "test")
    .group_by(["frame_distance_from_tackle"])
    .agg(
        n_frames = pl.len(),
        n_plays = pl.struct(["gameId", "playId"]).n_unique(),
        zoo_mse = pl.map_groups(
                exprs=["tackle_x_rel", "tackle_y_rel", "tackle_x_rel_pred_zoo", "tackle_y_rel_pred_zoo"],
                function=lambda list_of_series: calculate_mse(*list_of_series),
                returns_scalar=True,
            ).round(1),
        trfm_mse = pl.map_groups(
                exprs=["tackle_x_rel", "tackle_y_rel", "tackle_x_rel_pred_trfm", "tackle_y_rel_pred_trfm"],
                function=lambda list_of_series: calculate_mse(*list_of_series),
                returns_scalar=True,
            ).round(1),
    )
    .sort("frame_distance_from_tackle")
    .with_columns(
        trfm_perc_adv = ((pl.col("zoo_mse") - pl.col("trfm_mse"))*100 / pl.col("zoo_mse")).round(1)
    )
)

test_loss_by_frame_df

frame_distance_from_tackle,n_frames,n_plays,zoo_mse,trfm_mse,trfm_perc_adv
cat,u32,u32,f64,f64,f64
"""(-5, 0]""",11218,1122,21.9,4.5,79.5
"""(0, 5]""",11218,1122,18.5,5.7,69.2
"""(5, 10]""",10952,1121,15.6,5.8,62.8
"""(10, 15]""",10142,1045,14.3,6.6,53.8
"""(15, 20]""",9280,960,15.1,10.2,32.5
"""(20, 25]""",8366,876,18.7,16.4,12.3
"""(25, 30]""",7476,780,24.7,23.4,5.3
"""(30, 35]""",6742,704,31.5,29.6,6.0
"""(35, 40]""",5900,629,41.7,39.0,6.5


In [76]:
tracking_df = pl.read_parquet("../data/split_prepped_data/test_features.parquet")
tracking_df.sample(1)

gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,x,y,s,a,dis,o,dir,event,down,yardsToGo,distanceToGoal,weight,height_inches,is_ball_carrier,side,sx,sy,ox,oy,mirrored,play_origin_x,play_origin_y,x_rel,y_rel
i64,i64,i64,str,i64,str,i64,str,f64,f64,f64,f64,f64,f64,f64,str,i64,i64,i64,i64,i64,i64,i32,f64,f64,f64,f64,bool,f64,f64,f64,f64
2022090800,980,44875,"""Dion Dawkins""",7,"""2022-09-08 21:00:30.500000""",73,"""BUF""",40.09,17.37,1.13,0.11,0.12,311.29,306.25,null,1,10,68,320,77,0,1,0.66818,0.911282,0.659871,0.751379,true,46.63,51.2,-6.54,-33.83


In [77]:
play_df = pl.read_csv("../data/bdb_2024/plays.csv", null_values=["", "NA", "na", "nan", "NaN", "NAN"]).with_columns(
            distanceToGoal = (
                pl.when(pl.col("possessionTeam") == pl.col("yardlineSide"))
                .then(100 - pl.col("yardlineNumber"))
                .otherwise(pl.col("yardlineNumber"))
            )
        )
play_df.sample(10)

gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,distanceToGoal
i64,i64,i64,str,str,i64,i64,i64,str,str,str,i64,str,i64,i64,str,i64,i64,i64,i64,str,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,str,str,i64,str,i64
2022091812,2328,44995,"""Aaron Jones""","""(4:27) A.Jones right end to GB…",3,2,8,"""GB""","""CHI""","""GB""",26,"""4:27""",24,10,null,null,null,2,2,"""N""",36,"""I_FORM""",7,0.682615,0.972678,0.027322,-0.001227,0.001227,0.7513539,-0.679792,null,null,null,null,74
2022102304,2812,53454,"""Travis Etienne""","""(5:47) T.Etienne left end to J…",3,2,8,"""JAX""","""NYG""","""JAX""",6,"""5:47""",17,13,null,null,null,4,4,"""N""",104,"""SINGLEBACK""",8,0.683863,0.705749,0.294251,-0.002911,0.002911,-0.52788,-0.421138,null,null,null,null,94
2022101300,477,47856,"""David Montgomery""","""(6:29) D.Montgomery right guar…",1,2,5,"""CHI""","""WAS""","""WAS""",25,"""6:29""",0,0,null,null,null,2,2,"""N""",35,"""SINGLEBACK""",6,0.820639,0.549284,0.450716,-0.005515,0.005515,4.248687,-0.282591,null,null,null,null,25
2022091901,766,47836,"""Miles Sanders""","""(3:19) M.Sanders left tackle t…",1,1,10,"""PHI""","""MIN""","""PHI""",18,"""3:19""",7,0,null,null,null,3,3,"""N""",28,"""SINGLEBACK""",6,0.481833,0.757938,0.242062,-0.009083,0.009083,0.694226,-0.448804,null,null,null,null,82
2022100207,1824,43334,"""Derrick Henry""","""(2:08) (Shotgun) D.Henry up th…",2,1,10,"""TEN""","""IND""","""TEN""",26,"""2:08""",10,24,null,null,null,7,7,"""N""",36,"""SHOTGUN""",7,0.512253,0.13862,0.86138,-0.000213,0.000213,1.401139,0.054077,null,null,null,null,74
2022100911,3265,52630,"""Eno Benjamin""","""(11:22) (No Huddle, Shotgun) K…",4,2,14,"""ARI""","""PHI""","""PHI""",43,"""11:22""",10,17,"""C""",-3,null,16,16,"""N""",53,"""EMPTY""",5,0.898519,0.199254,0.800746,0.073119,-0.073119,1.975723,2.245066,null,null,null,null,43
2022110604,1212,53454,"""Travis Etienne""","""(10:51) (Shotgun) T.Etienne le…",2,1,10,"""JAX""","""LV""","""JAX""",25,"""10:51""",0,17,null,null,null,9,9,"""N""",85,"""SHOTGUN""",7,0.583805,0.129314,0.870686,0.000144,-0.000144,1.558347,0.707886,null,null,null,null,75
2022101000,3526,47807,"""Josh Jacobs""","""(7:25) (Shotgun) D.Carr pass s…",4,1,10,"""LV""","""KC""","""LV""",25,"""7:25""",30,23,"""C""",-5,null,13,13,"""N""",35,"""SHOTGUN""",6,0.729612,0.822256,0.177744,-0.001503,0.001503,1.389095,0.484537,null,null,null,null,75
2022103004,2782,46506,"""Dontrell Hilliard""","""(7:44) (Shotgun) D.Hilliard le…",4,3,5,"""TEN""","""HOU""","""TEN""",36,"""7:44""",3,17,null,null,0,0,0,"""N""",74,"""SHOTGUN""",7,0.938489,0.02981,0.97019,0.004062,-0.004062,0.842098,-1.470899,"""Offensive Holding""",null,53063,null,64


In [78]:
results_df.filter(pl.col("dataset_split") == "test").filter(pl.col("tackle_event") == "touchdown").sample(10)

gameId,playId,mirrored,frameId,tackle_x_rel_pred_trfm,tackle_y_rel_pred_trfm,dataset_split,ballCarrierNflId,ballCarrierName,tackle_frameId,tackle_event,tackle_x,tackle_y,tackle_x_rel,tackle_y_rel,tackle_event_enum,tackle_x_rel_pred_zoo,tackle_y_rel_pred_zoo,frame_distance_from_tackle
i64,i64,bool,i64,f32,f32,str,i64,str,i64,str,f64,f64,f64,f64,i64,f32,f32,cat
2022100202,775,true,47,11.92,-12.7,"""test""",52463,"""J.K. Dobbins""",47,"""touchdown""",111.71,12.15,13.22,-11.25,2,26.370001,-16.389999,"""(-5, 0]"""
2022092900,272,false,50,11.04,-2.27,"""test""",44860,"""Joe Mixon""",66,"""touchdown""",111.1,19.87,13.58,-3.47,2,11.48,-4.06,"""(15, 20]"""
2022103000,1636,false,8,7.88,-25.299999,"""test""",52423,"""Jerry Jeudy""",38,"""touchdown""",109.59,5.09,9.12,-27.5,2,5.04,-23.530001,"""(25, 30]"""
2022100909,2483,false,10,13.74,3.68,"""test""",53511,"""Dyami Brown""",8,"""touchdown""",109.73,49.66,5.89,1.7,2,7.31,5.78,"""(-5, 0]"""
2022103008,3406,false,49,19.99,26.969999,"""test""",47836,"""Miles Sanders""",56,"""touchdown""",110.68,53.5,16.86,25.73,2,15.57,27.309999,"""(5, 10]"""
2022103003,180,false,5,11.65,4.63,"""test""",44947,"""Jamaal Williams""",41,"""touchdown""",110.52,22.67,15.19,-1.09,2,11.2,7.26,"""(35, 40]"""
2022100202,775,false,29,11.08,8.02,"""test""",52463,"""J.K. Dobbins""",47,"""touchdown""",111.71,41.15,13.22,11.25,2,14.12,10.84,"""(15, 20]"""
2022100911,1200,true,1,1.05,1.0,"""test""",52461,"""Jalen Hurts""",18,"""touchdown""",109.69,26.96,1.43,-0.37,2,1.79,2.62,"""(15, 20]"""
2022100202,775,true,22,9.1,-3.03,"""test""",52463,"""J.K. Dobbins""",47,"""touchdown""",111.71,12.15,13.22,-11.25,2,9.97,-4.66,"""(20, 25]"""


In [81]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go



def animate_play(tracking_df: pl.DataFrame, play_df: pl.DataFrame, results_df: pl.DataFrame, gameId: int, playId: int, mirrored: bool):
    """
    Animate a play from the tracking data and the results of the models.
    """
    mvmt_df = tracking_df.filter((pl.col("gameId") == gameId) & (pl.col("playId") == playId) & (pl.col("mirrored") == mirrored)).to_pandas().round(2)
    play_df = play_df.filter((pl.col("gameId") == gameId) & (pl.col("playId") == playId)).to_pandas()
    model_results_df = results_df.filter((pl.col("gameId") == gameId) & (pl.col("playId") == playId) & (pl.col("mirrored") == mirrored)).to_pandas()

    mvmt_df["side"] = mvmt_df["side"].replace({1: "OFF", -1: "DEF"})
    # display(mvmt_df.sample(3), play_df, model_results_df)

    # get some info
    distToGoal = play_df["distanceToGoal"].values[0]
    down = play_df["down"].values[0]
    yards_to_go = play_df["yardsToGo"].values[0]
    play_description = play_df["playDescription"].values[0]
    off_club = mvmt_df.loc[mvmt_df["side"] == "OFF", "club"].values[0]
    def_club = mvmt_df.loc[mvmt_df["side"] == "DEF", "club"].values[0]

    mvmt_y_min = mvmt_df["x"].min()
    mvmt_y_max = mvmt_df["x"].max()

    tkl_x, tkl_y = model_results_df[["tackle_x", "tackle_y"]].values[0]
    # origin_x, origin_y = mvmt_df[["play_origin_x", "play_origin_y"]].values[0]
    # zoo_tkl_x_rel, zoo_tkl_y_rel = model_results_df[["tackle_x_rel_pred_zoo", "tackle_y_rel_pred_zoo"]].values[0]
    # trfm_tkl_x_rel, trfm_tkl_y_rel = model_results_df[["tackle_x_rel_pred_trfm", "tackle_y_rel_pred_trfm"]].values[0]
    # zoo_tkl_x, zoo_tkl_y = origin_x + zoo_tkl_x_rel, origin_y + zoo_tkl_y_rel
    # trfm_tkl_x, trfm_tkl_y = origin_x + trfm_tkl_x_rel, origin_y + trfm_tkl_y_rel

    # print(los, down, yards_to_go, play_description)
    # print(y_min, y_max, (tkl_x, tkl_y), (zoo_tkl_x, zoo_tkl_y), (trfm_tkl_x, trfm_tkl_y))

    
    # set some things
    mvmt_df["size"] = 2
    mvmt_df["text_color"] = "black"
    # mvmt_df.loc[mvmt_df["side"] == "BALL", "color1"] = "#fcca53"
    mvmt_df.loc[mvmt_df["side"] == "OFF", "color1"] = "black"
    mvmt_df.loc[mvmt_df["side"] == "DEF", "color1"] = "white"
    # mvmt_df.loc[mvmt_df["side"] == "BALL", "size"] = 0

    # Different symbols for different positions
    mvmt_df["symbol"] = 1
    mvmt_df.loc[mvmt_df["is_ball_carrier"] == 1, "symbol"] = 2
    # mvnt.loc[mvnt["side"] == "BALL", "symbol"] = "diamond_0"
    symbol_map = {1: "circle", 2: "hexagon"}

    # Data to display on hover
    hover_data = {
        "displayName": True,
        "club": True,
        "side": True,
        "jerseyNumber": True,
        "is_ball_carrier": True,
        "color1": False,
        "symbol": True,
        "frameId": False,
        "x": True,
        "y": True,
        "sx": False,
        "sy": False,
        "size": False,
    }

    X_LEFT = 0
    X_MIDDLE = 160 / 6.0
    X_RIGHT = 160 / 3.0
    Y_MIN = 0
    Y_MAX = 120
    Y_MIDDLE = 60
    fig = px.scatter(
        mvmt_df,
        x="y",
        y="x",
        animation_frame="frameId",
        animation_group="nflId",
        hover_name="displayName",
        hover_data=hover_data,
        text="jerseyNumber",
        width=1000,
        height=1200,
        # range_x=[-160 / 6.0, 160 / 6.0],
        range_x=[X_LEFT, X_RIGHT],
        size="size",
        size_max=15,
        color="color1",  # Ensure color column is set correctly
        color_discrete_sequence=["#FF69B4", "#39FF14"],
        opacity=0.8,
        symbol="symbol",
        symbol_map=symbol_map,
    )

    # Add marker for tackle location
    fig.add_trace(
        go.Scatter(
            x=[tkl_y],
            y=[tkl_x],
            mode="markers",
            marker=dict(color="yellow", size=12, symbol="x"),
            hoverinfo="none",
            showlegend=False,
            opacity=0.8,
        )
    )

    # Add line of scrimmage
    los = Y_MAX-10-distToGoal
    fig.add_shape(
        type="line", x0=X_LEFT, y0=los, x1=X_RIGHT, y1=los, line=dict(color="rgba(137, 207, 240, 0.2)", width=3, dash="dash")
    )
    # Add yards to go line
    fig.add_shape(
        type="line",
        x0=X_LEFT,
        y0=los + yards_to_go,
        x1=X_RIGHT,
        y1=los + yards_to_go,
        line=dict(color="rgba(255, 255, 0, 0.2)", width=3, dash="dash"),
    )
    # Add border to the field
    fig.add_shape(
        type="rect", x0=X_LEFT, y0=Y_MIN, x1=X_RIGHT, y1=Y_MAX, line=dict(color="rgba(255, 255, 255, 0.5)", width=10)
    )
    # Add the path traces to the figure first to place them in the background
    # for trace in path_traces:
    #     fig.add_trace(trace)

    # set play speed
    frame_duration = 100
    for button in fig.layout.updatemenus[0].buttons:
        button["args"][1]["frame"]["duration"] = frame_duration
    # set aspect ratio
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    # background color
    fig.update_layout(paper_bgcolor="#333333", plot_bgcolor="#363636", font_color="white", font_size=14)
    # turn off axis
    fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False)
    # grid line thickness
    fig.update_yaxes(
        showgrid=True,
        gridwidth=3,
        gridcolor="rgba(237, 234, 222, 0.1)",
        linewidth=0,
        linecolor="rgba(0, 0, 0, 0.01)",
        mirror=True,
        showticklabels=False,
    )
    # set y axis range
    fig.update_yaxes(range=[mvmt_y_min, mvmt_y_max])
    # set yaxes ticks to 10 yards
    # fig.update_yaxes(tick0=0, dtick=10)
    # text size
    fig.update_layout(uniformtext_minsize=2, uniformtext_mode="hide")
    # hide legend
    fig.update_layout(showlegend=False)
    # text color of jersey numbers
    fig.update_traces(textfont=dict(family="Tahoma", size=12, color=mvmt_df["text_color"]))
    fig.update_traces(marker_line_width=0)

    # hide x and y labels
    fig.update_xaxes(title_text="")
    fig.update_yaxes(title_text="")
    
    # add hash marks
    for y_loc in range(Y_MIN+10, Y_MIN-10+1, 1):
        if y_loc % 10 == 0:
            ydln = y_loc-10 if y_loc <= 60 else 110 - y_loc
            ydln_txt = str(ydln) if ydln != 0 else "E Z"
            fig.add_shape(
                type="line", x0=X_LEFT, y0=y_loc, x1=X_RIGHT, y1=y_loc, line=dict(color="white", width=2), opacity=0.05
            )
            fig.add_annotation(
                x=X_LEFT+4, y=y_loc, text=ydln_txt, showarrow=False, font=dict(color="white", size=60), textangle=90, opacity=0.05
            )
            fig.add_annotation(
                x=X_RIGHT-4, y=y_loc, text=ydln_txt, showarrow=False, font=dict(color="white", size=60), textangle=270, opacity=0.05
            )
        elif y_loc % 5 == 0:
            fig.add_shape(
                type="line", x0=X_LEFT, y0=y_loc, x1=X_RIGHT, y1=y_loc, line=dict(color="white", width=1), opacity=0.05
            )
        else:
            fig.add_shape(
                type="rect", x0=X_MIDDLE-9.5, y0=y_loc, x1=X_MIDDLE-8.5, y1=y_loc, line=dict(color="white", width=3), opacity=0.05
            )
            fig.add_shape(
                type="rect", x0=X_MIDDLE+9.5, y0=y_loc, x1=X_MIDDLE+8.5, y1=y_loc, line=dict(color="white", width=3), opacity=0.05
            )

    fig.add_annotation(x=X_MIDDLE, y=Y_MAX-5, text=off_club, showarrow=False, font=dict(color="white", size=90), opacity=0.2)
    fig.add_annotation(x=X_MIDDLE, y=Y_MIDDLE+2, text="SŪMER", showarrow=False, font=dict(color="white", size=50), opacity=0.05)
    fig.add_annotation(x=X_MIDDLE, y=Y_MIDDLE-2, text="SPORTS", showarrow=False, font=dict(color="white", size=50), opacity=0.05)

    # Add play description
    if play_description is not None:
        # make list of 100 character slices
        play_desc_list = [play_description[i : i + 100] for i in range(0, len(play_description), 100)]
        for i, play_desc_txt in enumerate(play_desc_list):
            text_y_loc = mvmt_y_min - 10 - 5 * i
            fig.add_annotation(
                x=X_MIDDLE, y=text_y_loc, text=play_desc_txt, showarrow=False, font=dict(color="white", size=14), opacity=0.8
            )

    # set title
    # offense = play_info["offense"].values[0].upper()
    # defense = play_info["defense"].values[0].upper()
    # down = int(play_info["down"].values[0])
    # yards_to_go = int(play_info["yards_to_go"].values[0])
    # quarter = int(play_info["quarter"].values[0])
    # game_clock = play_info["game_clock"].values[0]
    fig.update_layout(
        title=f"{gameId} {playId} | {off_club} vs {def_club} | Down: {down} | YTG: {yards_to_go} | DTG: {distToGoal}",
        font_size=12,
        title_x=0.5,
        title_y=0.98,
    )

    return fig

animate_play(tracking_df, play_df, results_df, 2022091812, 814, True)